In [47]:
import pymongo

myclient = pymongo.MongoClient("mongodb://xiaotianxt.cn:27017")
mydb = myclient["lianjia"]
mycol = mydb["xiaoqu"]

tocol = mydb["location"]

In [3]:
print(mycol.find_one())

{'_id': ObjectId('60a2a11b0bb7f9259bb37374'), 'city': '北京', 'date': '20210518', 'district': '亦庄开发区', 'area': '马驹桥', 'xiaoqu': '珠江逸景', 'price': 27736, 'sale': 96}


In [80]:
rem = []
for item in xiaoqus:
    rem.append(item)
    if len(rem) == 10:
        for item, location in zip(rem, convert(rem)):
            lat, lon = location
            tocol.insert_one({
                'city': item['city'],
                'district': item['district'],
                'area': item['area'],
                'xiaoqu': item['xiaoqu'],
                'lat': lat,
                'lon': lon
            })
        rem = []

In [22]:

def convert(items):
    import requests
    url = r'https://restapi.amap.com/v3/geocode/geo'
    response = requests.get(url,params={
        "key":"3098971a0dd91ecd75778af52bb4d3d0",
        "batch": "true",
        "output": "json",
        "address": "|".join(list([f"{item['city']}市{item['district']}区{item['area']}{item['xiaoqu']}" for item in items])),
        "city": "北京"
    })
    ret = response.json()
    return [(float(item['location'].split(',')[0]), float(item['location'].split(',')[1])) for item in ret['geocodes']]

北京 亦庄开发区 马驹桥 天鹅堡 116.536217 39.749356
北京 亦庄开发区 马驹桥 金地格林6 116.541859 39.752069
北京 亦庄开发区 马驹桥 新海北里 116.541884 39.756792
北京 亦庄开发区 马驹桥 样本 116.549526 39.756462
北京 亦庄开发区 马驹桥 新海南里 116.541142 39.754057
北京 亦庄开发区 马驹桥 合生世界花园 116.554998 39.739561
北京 亦庄开发区 马驹桥 珠江萨尔斯郡 116.544998 39.742401
北京 大兴 瀛海 德茂佳苑 116.438908 39.777337
北京 大兴 瀛海 紫宸苑 116.456526 39.76258
北京 大兴 瀛海 瀛海家园融园D区 116.434715 39.760667


In [68]:
help(mycol.distinct)

Help on method distinct in module pymongo.collection:

distinct(key, filter=None, session=None, **kwargs) method of pymongo.collection.Collection instance
    Get a list of distinct values for `key` among all documents
    in this collection.
    
    Raises :class:`TypeError` if `key` is not an instance of
    :class:`basestring` (:class:`str` in python 3).
    
    All optional distinct parameters should be passed as keyword arguments
    to this method. Valid options include:
    
      - `maxTimeMS` (int): The maximum amount of time to allow the count
        command to run, in milliseconds.
      - `collation` (optional): An instance of
        :class:`~pymongo.collation.Collation`. This option is only supported
        on MongoDB 3.4 and above.
    
    The :meth:`distinct` method obeys the :attr:`read_preference` of
    this :class:`Collection`.
    
    :Parameters:
      - `key`: name of the field for which we want to get the distinct
        values
      - `filter` (optional)

In [77]:
xiaoqus = [item['_id'] for item in mycol.aggregate([{"$group":{"_id":{"city":"$city", "district":"$district", "area":"$area", "xiaoqu":"$xiaoqu"}}}])]